In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
import xgboost as xgb
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [154]:
# read df full csv
df_full=pd.read_csv('../../data/consolidated data/df_full_01November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp'
)
df_full.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.23733,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.76267,39.23733,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [155]:
# Desfase de perdida de TPH para eliminar las causas de las perdidas
for i in range(1,6):
    df_full[f"loss of TPH_{i}"]=df_full["loss of TPH"].shift(-i)

df_full["loss of TPH"]=df_full.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or \
(x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)

In [238]:
# TPH promedio según HH charge cell
TPH_historico=df_full[["HH charge cell","TPH"]].groupby("HH charge cell").mean().reset_index()
TPH_historico

,HH charge cell,TPH
0,660.000000,4349.634608
1,660.002292,4295.012475
2,660.004585,4294.244128
3,660.006877,4304.960976
4,660.009169,4307.145630
...,...,...
2971,1012.710324,4058.039942
2972,1012.780870,4066.605017
2973,1012.851417,4029.256336
2974,1012.921963,4015.668759


In [235]:
def cost_TPH(df,cc):
    df["distance"]=(df["HH charge cell"]-cc).abs()
    x=TPH_historico[TPH_historico["distance"]==TPH_historico.distance.min()]["TPH"].values[0]
    return x
    

In [236]:
df=df_full[(df_full["loss of TPH"]==0) & (df_full["Rec HH CC less than HH CC"]==1) & (df_full["greater than Rec HH CC"]==1)].dropna()
df.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-08 16:24:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.51,1.65,0.39,0.01,19.8478,112.1727,15.7825,0.03,63.117787,787.21956,19641.296793,4079.304899,73.877412,1439.921189,8.249531,74.0,720.0,800.0,4100.0,3667.648521,66.705534,2.0,20.695101,12.78044,67.21956,0,0,774.588893,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1.0,1.0,0,0,0


In [240]:
df_metric=df[["charge cell","TPH","HH TPH","recommendation","HH charge cell"]]
df_metric["TPH historic"]=df["recommendation"].apply(lambda x: cost_TPH(TPH_historico,x))
df_metric["cost TPH"]=df_metric["TPH"]-df_metric["TPH historic"]
df_metric

,charge cell,TPH,HH TPH,recommendation,HH charge cell,TPH historic,cost TPH
Timestamp,,,,,,,
2020-02-08 16:24:00,787.219560,4079.304899,4100.0,774.588893,800.0,3989.669743,89.635156
2020-02-08 16:25:00,787.558897,4087.649222,4100.0,774.588893,800.0,3989.669743,97.979479
2020-02-08 16:26:00,788.579123,4080.193127,4100.0,774.588893,800.0,3989.669743,90.523384
2020-02-08 16:27:00,790.698064,4093.222860,4100.0,774.588893,800.0,3989.669743,103.553117
2020-02-08 16:28:00,791.900323,4084.807274,4100.0,774.588893,800.0,3989.669743,95.137531
...,...,...,...,...,...,...,...
2022-02-28 09:54:00,914.137054,4100.673792,4120.0,907.966856,950.0,3803.968354,296.705438
2022-02-28 09:55:00,919.262837,4090.384509,4120.0,907.966856,950.0,3803.968354,286.416155
2022-02-28 09:56:00,922.912778,4082.137827,4120.0,918.208691,950.0,3778.918581,303.219246


In [241]:
df_metric["cost TPH"].sum()

7829440.7406747155

In [195]:
inicio_periodo=parse('2021-01-11 15:47:00')
fin_periodo=inicio_periodo+timedelta(minutes=10)
data=df_full.copy()[inicio_periodo:fin_periodo]
data.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-11 15:47:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.01,1.28,0.11,0.0,15.7262,109.25,16.2088,0.0,62.099123,901.77079,18654.058453,3699.382787,77.902083,1051.44581,7.968904,77.75,890.0,970.0,3700.0,3607.42529,67.736528,2.0,0.617213,68.22921,11.77079,0,0,788.670246,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,0,0,0


In [196]:
titulos=tuple(["TPH & HH TPH", "CC, recommendation HH CC, HH CC & LL CC","granulometry","SPI","loss of TPH","diff. HH TPH between TPH",
"greater than HH CC","greater than Rec HH CC","Rec HH CC less than HH CC","less than LL CC","start loss tph","end loss tph","water","speed","solid percentage"])
fig = make_subplots(
    rows=5, cols=3,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     #shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  



fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], ##FF6511
                    mode='lines',
                    name="LL charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], ##FF6511
                    mode='lines',
                    name="HH charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data["recommendation"], ##FF6511
                    mode='lines',
                    name="recommendation HH CC",line=dict(width=1,color="black"),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=1, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=5, col=3)  


for i in range(1,6):
   for j in range(1,4):
        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.add_trace(go.Scatter(x=data.index, y=data[ 'water'], ##FF6511
                    mode='lines',
                    name= 'water',line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data[  'loss of TPH'], ##FF6511
                    mode='lines',
                    name=  'loss of TPH',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  


fig.add_trace(go.Scatter(x=data.index, y=data[ "dif_TPH_HH"], ##FF6511
                    mode='lines',
                    name=  "dif_TPH_HH",line=dict(width=3, ),legendgroup = '1'),row=2, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'less than LL CC'], ##FF6511
                    mode='lines',
                    name=  'less than LL CC',line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'start loss tph'], ##FF6511
                    mode='lines',
                    name=  'start loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'end loss tph'], ##FF6511
                    mode='lines',
                    name=  'end loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=3)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than Rec HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than rec HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ "Rec HH CC less than HH CC"], ##FF6511
                    mode='lines',
                    name=  "Rec HH CC less than HH CC",line=dict(width=3, ),legendgroup = '1'),row=3, col=3)  
fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()